In [ ]:
def make_playlist(name,description, public = False, collaborative=False):
    playlist = spotify.user_playlist_create(my_id, name, public=public, collaborative=collaborative, description=description)
    playlist.pop('owner',None)
    playlist.pop('tracks',None)
    playlist = pd.DataFrame({k: [v] for k, v in playlist.items()})
    global playlists
    playlists = pd.concat([playlist, playlists])
    return(playlists)

def edit_playlist_details(playlist_id, user = my_id, name=None, public=None, collaborative=None, description=None):
    spotify.user_playlist_change_details(user, playlist_id, name=name, public=public, collaborative=collaborative, description=description)
    global playlists
    if name != None:
        playlists.loc[playlists['id'] == playlist_id,['name']] = name
    if description != None:
        playlists.loc[playlists['id'] == playlist_id,['description']] = description
    if collaborative != None:
        playlists.loc[playlists['id'] == playlist_id,['collaborative']] = collaborative
    if public != None:
        playlists.loc[playlists['id'] == playlist_id,['public']] = public
    return(playlists)

def delete_playlist(playlist_id, user = my_id):
    spotify.user_playlist_unfollow(user, playlist_id)
    global playlists
    playlists = playlists[~(playlists['id'] == playlist_id)]
    return(playlists)

def get_playlist(playlist_id, with_tracks = True):
    playlist = spotify.playlist(playlist_id)
    playlist.pop('owner',None)
    playlist.pop('tracks',None)
    global albums, artists, playlists, tracks, playlist_tracks
    playlist = pd.DataFrame({k: [v] for k, v in playlist.items()})
    playlists = pd.concat([playlists, playlist]).drop_duplicates(subset = 'id', keep = "last")
    
    if with_tracks == True:
        offset = 0
        total = 10000
        temp = []
        while offset<total:
            track_data = spotify.user_playlist_tracks(user=my_id,playlist_id=playlist_id, offset = offset)
            temp = temp + [clean_track_data(x['track']) for x in track_data['items']]
            if offset == 0:
                total = track_data['total']
            offset += 100
        if temp != []:
            tracks_instance = pd.concat([item[0] for item in temp])
            tracks = pd.concat([tracks, tracks_instance]).drop_duplicates(subset='Track ID', keep='last')
            tracks_instance['Playlist ID'] = playlist_id
            tracks_instance['Position Number'] = np.arange(len(tracks_instance))
        
            if playlist_tracks.size != 0:
                playlist_tracks = pd.concat([playlist_tracks[playlist_tracks['Playlist ID'] != playlist_id], tracks_instance[['Track ID', 'Playlist ID', 'Position Number']]])
            else:
                playlist_tracks = tracks_instance[['Track ID', 'Playlist ID', 'Position Number']]
            albums = pd.concat([albums] + [item[1] for item in temp]).drop_duplicates(subset='Album ID', keep='last')
            artists = pd.concat([artists] + [item[2] for item in temp]).drop_duplicates(subset='Artist ID', keep='last')
        
        return(playlist_tracks)
    
    else:
        return(playlists)

In [ ]:
def edit_playlist_tracks(playlist_id, new_tracks):
    
    get_playlist(playlist_id = playlist_id, with_tracks = True)

    new_tracks_df = pd.DataFrame()
    new_tracks_df['Track ID'] = pd.Series(new_tracks)
    new_tracks_df['New Position Number'] = np.arange(len(new_tracks))
    
    if playlist_tracks.size != 0:
        current_playlist = playlist_tracks[playlist_tracks['Playlist ID'] == playlist_id]
        new_tracks_df = new_tracks_df.merge(current_playlist, how = 'outer', on = 'Track ID')
    else:
        current_playlist = playlist_tracks
        new_tracks_df['Position Number'] = pd.NA
        new_tracks_df['Playlist ID'] = pd.NA 
    
    remove_tracks = list(new_tracks_df[new_tracks_df['New Position Number'].isna()]['Track ID'])
    if len(remove_tracks) != 0:
        spotify.playlist_remove_all_occurrences_of_items(playlist_id = playlist_id, items = remove_tracks)
    
    new_tracks_df = new_tracks_df[~new_tracks_df['New Position Number'].isna()].sort_values(by = 'Position Number')
    
    add_tracks = list(new_tracks_df[new_tracks_df['Position Number'].isna()]['Track ID'])
    if len(add_tracks) != 0:
        i = 0
        while i < len(add_tracks) // 50 + 1:
            temp_tracks_to_add = add_tracks[50*i:50*(i+1)]
            spotify.playlist_add_items(playlist_id = playlist_id, items = temp_tracks_to_add, position=50*i)
            i += 1
        
    new_tracks_df['Position Number'] = np.arange(len(new_tracks))
    new_tracks_df = new_tracks_df.sort_values(by = 'New Position Number')

    for i in range(len(new_tracks_df.index)):
        new_pos = int(new_tracks_df.loc[i, 'New Position Number'])
        pos = int(new_tracks_df.loc[i, 'Position Number'])
        if new_pos != pos:
            spotify.playlist_reorder_items(playlist_id = playlist_id, range_start=pos, insert_before=new_pos, range_length=1)
            new_tracks_df.loc[(new_tracks_df['Position Number'] < pos) & (new_tracks_df['Position Number'] >= new_pos),'Position Number'] += 1
            new_tracks_df.loc[(new_tracks_df['New Position Number'] == new_pos),'Position Number'] = new_pos
    
    get_playlist(playlist_id = playlist_id, with_tracks = True)
    current_playlist = playlist_tracks[playlist_tracks['Playlist ID'] == playlist_id]
    
    return(current_playlist)